# Reservoir Geomechanics EDX Course (GEOPHYSX0001)- Homework 01

The Stanford Online Reservoir Geomechanics course is available at [EDX](https://courses.edx.org/courses/course-v1:StanfordOnline+GEOPHYSX0001+2T2020/course/). You can find more information about the course [here](https://online.stanford.edu/courses/soeees-ygeoresgeo202-reservoir-geomechanics).<br>

This file contains answers to homework number one. One should follow the questions available in `Reservoir_Geomechanics_MOOC_HW1_2020.pdf` to understand the answers provided by the present file.<br>

In [1]:
import os
import numpy as np

In [2]:
# Folder that stores the data provided by EDX.
DATA = "data"

---

## Table of Contents

* [Read the Data](#read-data)
* [I. Compute the Overburden Stress and the Overburden Gradient](#compute-overburden)
* [II. Compute Porosity From Density Measurements](#compute-porosity)

---

## Read the Data <a class="anchor" id="read-data"></a>

The following files are provided by EDX, and are stored in the `data` folder:
* `Barnett_density_data.txt`
* `GOM_offshore_density_data.txt`

Opening the files using a text editor, it is possible to see that both have the same structure. We can see that:<br>

1 - The first line holds the headers.<br>
2 - The second line is blank.<br>
3 - The third line is the first line that contains data.

### Barnett data

In [8]:
barnett = np.loadtxt(os.path.join(DATA, "Barnett_density_data.txt"), skiprows=2)

In [9]:
# Take a look ate the first 10 lines:
barnett[0:9]

array([[100.5   ,   1.8778],
       [101.    ,   1.8709],
       [101.5   ,   1.8317],
       [102.    ,   1.8363],
       [102.5   ,   1.85  ],
       [103.    ,   1.8441],
       [103.5   ,   1.8299],
       [104.    ,   1.8303],
       [104.5   ,   1.8555]])

### GOM data

In [10]:
GOM = np.loadtxt(os.path.join(DATA, "GOM_offshore_density_data.txt"), skiprows=2)

In [11]:
# Take a look ate the first 10 lines:
GOM[0:9]

array([[3.515e+03, 2.154e+00],
       [3.516e+03, 2.126e+00],
       [3.517e+03, 2.098e+00],
       [3.518e+03, 2.063e+00],
       [3.519e+03, 2.084e+00],
       [3.520e+03, 2.114e+00],
       [3.521e+03, 2.128e+00],
       [3.522e+03, 2.166e+00],
       [3.523e+03, 2.128e+00]])

## I. Compute the overburden stress and the overburden gradient <a class="anchor" id="compute-overburden"></a>

## II. Compute porosity from density measurements <a class="anchor" id="compute-porosity"></a>